In [1]:
import numpy as np 
from numpy import genfromtxt 
import matplotlib.pyplot as plt 

In [3]:
data = genfromtxt('advertising.csv', delimiter=',', skip_header=1)

In [5]:
data.shape 

(200, 4)

In [6]:
X = data[:, :3]
X[:5]

array([[230.1,  37.8,  69.2],
       [ 44.5,  39.3,  45.1],
       [ 17.2,  45.9,  69.3],
       [151.5,  41.3,  58.5],
       [180.8,  10.8,  58.4]])

In [7]:
y = data[:, 3]

In [8]:
y[:5]

array([22.1, 10.4, 12. , 16.5, 17.9])

In [9]:
thetas = np.asarray([[1.16270837], [-0.81960489], [1.39501033], [0.29763545]])
thetas

array([[ 1.16270837],
       [-0.81960489],
       [ 1.39501033],
       [ 0.29763545]])

In [10]:
N = X.shape[0]
N

200

In [ ]:
def stochastic_gradient_descent(X_b, y, n_epochs=50, learning_rate=0.00001):

    # khởi tạo giá trị tham số
    # thetas = np.random.randn(4, 1) # thực tế các bạn nên bỏ comment dòng này
    thetas = np.asarray([[1.16270837], [-0.81960489],  [1.39501033],  [0.29763545]])

    thetas_path = [thetas] # chuyển thành array
    losses = []

    for epoch in range(n_epochs):
        for i in range(N):
            random_index = i 

            xi = X_b[random_index:random_index+1]
            yi = y[random_index:random_index+1]

            # tính output
            oi = xi.dot(thetas)

            # tính loss li
            li = (oi - yi)*(oi - yi) / 2

            # tính gradient cho loss
            g_li = (oi - yi)

            # tính gradient
            gradients = xi.T.dot(g_li)

            # update giá trị theta
            thetas = thetas - learning_rate*gradients

            # logging
            thetas_path.append(thetas)
            losses.append(li[0][0])

    return thetas_path, losses

sgd_theta, losses = stochastic_gradient_descent(X_b, y, n_epochs=50, learning_rate=0.01 )

# in loss cho 500 sample đầu
x_axis = list(range(500))
plt.plot(x_axis,losses[:500], color="r")
plt.show()

In [13]:
X_b = np.c_[np.ones((N, 1)), X]
X_b.shape

(200, 4)

In [14]:
losses, log_theta = sgd(X, y, n_epochs=50, learning_rate=0.00001)

ValueError: shapes (1,3) and (4,1) not aligned: 3 (dim 1) != 4 (dim 0)